## In this notebook, I analyze [this Kaggle dataset](https://www.kaggle.com/rikdifos/credit-card-approval-prediction). The dataset consists of demographic and financial data for accounts at a bank (unspecified) along with a credit history of that account. The same customer at the bank may have multiple accounts attached to them. The goal of the notebook is to clean up the data and construct a label for each customer using all of their accounts as good or bad credit. This notebook will later be used to build a model to predict the credit score of customers using their demographic and financial data, to quantify the risk of opening a credit account.

### Data Cleaning

In [3]:
import sys
import pickle
import itertools
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn

Let's begin by loading our two dataframes.

In [95]:
with open("application_record.csv", "r") as app_data:
    app_df = pd.read_csv(app_data)
    

In [130]:
print(app_df)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008805           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
4       5008809           F            N               Y             0   
...         ...         ...          ...             ...           ...   
438552  6840104           M            N               Y             0   
438553  6840222           F            N               N             0   
438554  6841878           F            N               N             0   
438555  6842765           F            N               Y             0   
438556  6842885           F            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

In [97]:
with open("credit_record.csv", "r") as credit_data:
    credit_df = pd.read_csv(credit_data)

In [98]:
print(credit_df)

              ID  MONTHS_BALANCE STATUS
0        5001711               0      X
1        5001711              -1      0
2        5001711              -2      0
3        5001711              -3      0
4        5001712               0      C
...          ...             ...    ...
1048570  5150487             -25      C
1048571  5150487             -26      C
1048572  5150487             -27      C
1048573  5150487             -28      C
1048574  5150487             -29      C

[1048575 rows x 3 columns]


Notice that the application data has duplicates, the ID key doesn't uniquely identify a customer. We need to identify the cause of the duplication. It is possible these are multiple accounts for the same peron, and it is also possible these multiple accounts have different credit histories. To account for this, we create some functions that help us identify duplicate data. We first create a function that takes an index and returns all duplicates below it in the dataframe, assuming contiguous duplicates. We also create a function that returns the next index that isn't a duplicate of the given one.

In [248]:
def DuplicateList(df, index, col_list):
    '''Parameters are a pandas dataframe, an index number and a list of column names. 
       Returns a list of all index values greater than or equal to the input index that have identical data under the given columns.
       Assumes the duplicated data in the dataframe is contiguous.'''
    
    Duplicates = [index]
    df_null = df.isnull()
    #Testing if the latest entry checked was a duplicate.
    is_Latest_Dupe = True
    for i in [x for x in df.index.values if x > index]:
        if is_Latest_Dupe:
            Dupe_Status = True
            for col in col_list:
                if df_null.loc[index, col] and df_null.loc[i, col]:
                    Dupe_Status=Dupe_Status
                else: 
                    Dupe_Status = Dupe_Status and (df.loc[index, col]==df.loc[i, col])
            if Dupe_Status:
                Duplicates.append(i)
            is_Latest_Dupe = is_Latest_Dupe and Dupe_Status
        elif not is_Latest_Dupe:
            break
    
    return Duplicates

Let's test this on some of the values we can see in our dataframe. 

In [270]:
#Assigning variables to the columns of our app_df dataframe

columns = app_df.columns.values.tolist()
columns_remaining = [x for x in columns if x!='ID']

print(DuplicateList(df=app_df, index=0, col_list = columns_remaining))

[0, 1]


In [271]:
for i in [2, 3, 4, 438552, 438556]:
    print(DuplicateList(df=app_df, index=i, col_list=columns_remaining))
print(DuplicateList(app_df, index=1315, col_list=columns_remaining))
print(app_df.iloc[[1315, 1316],:])

[2]
[3, 4, 5, 6]
[4, 5, 6]
[438552]
[438556]
[1315, 1316, 1317]
           ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
1315  5010298           F            N               Y             0   
1316  5010299           F            N               Y             0   

      AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
1315          135000.0  Commercial associate  Secondary / secondary special   
1316          135000.0  Commercial associate  Secondary / secondary special   

     NAME_FAMILY_STATUS  NAME_HOUSING_TYPE  DAYS_BIRTH  DAYS_EMPLOYED  \
1315            Married  House / apartment      -14918          -1866   
1316            Married  House / apartment      -14918          -1866   

      FLAG_MOBIL  FLAG_WORK_PHONE  FLAG_PHONE  FLAG_EMAIL OCCUPATION_TYPE  \
1315           1                0           0           0     Sales staff   
1316           1                0           0           0     Sales staff   

      CNT_FAM_MEMBERS  
1315 

In [275]:
def NextNonDuplicate(df, index, col_list):
    '''Arguments are a dataframe with contiguous duplicate data, an index value and list of columns to compare
        for duplication.
       Returns the index of the next datapoint that isn\'t a duplicate of the data at index.'''
    if index <0:
        index = len(df.index.values)+index
    if index >=len(df.index.values)-1:
        return None
    elif DuplicateList(df, index, col_list)[-1] >= len(df.index.values)-1:
        return None
    else:
        return [DuplicateList(df, index, col_list)[-1]+1, df.loc[DuplicateList(df, index, col_list)[-1]+1, "ID"]]


We can now use the drop_duplicates method in pandas to remove duplicates from app_df, but this would create some issues. The account id's in the application data are linked to the account id's in the credit data. If we remove duplicates first, we will potentially lose information. Instead, we will adopt a different strategy. We identify the accounts that are in both dataframes and only condense each dataframe to those accounts for which we have both application data and credit data. This is a natural procedure to make here, as these are the only data points we can actually use to fit a model, as for other accounts, we either lack feature data or data to construct a label. 

In [167]:
#Boolean series that tracks indices of app_df whose "ID" keys are also in credit_df

app_boolean = app_df["ID"].isin(credit_df["ID"])

In [171]:
app_df_only_including_overlap = app_df.loc[app_boolean]
print(app_df_only_including_overlap)

             ID CODE_GENDER FLAG_OWN_CAR FLAG_OWN_REALTY  CNT_CHILDREN  \
0       5008804           M            Y               Y             0   
1       5008805           M            Y               Y             0   
2       5008806           M            Y               Y             0   
3       5008808           F            N               Y             0   
4       5008809           F            N               Y             0   
...         ...         ...          ...             ...           ...   
434808  5149828           M            Y               Y             0   
434809  5149834           F            N               Y             0   
434810  5149838           F            N               Y             0   
434811  5150049           F            N               Y             0   
434812  5150337           M            N               Y             0   

        AMT_INCOME_TOTAL      NAME_INCOME_TYPE            NAME_EDUCATION_TYPE  \
0               427500.0      

Let us do the same for the credit data and only keep the overlap.

In [172]:
credit_boolean = credit_df["ID"].isin(app_df["ID"])

In [173]:
credit_df_only_including_overlap = credit_df.loc[credit_boolean]
print(credit_df_only_including_overlap)

              ID  MONTHS_BALANCE STATUS
92938    5008804               0      C
92939    5008804              -1      C
92940    5008804              -2      C
92941    5008804              -3      C
92942    5008804              -4      C
...          ...             ...    ...
1048570  5150487             -25      C
1048571  5150487             -26      C
1048572  5150487             -27      C
1048573  5150487             -28      C
1048574  5150487             -29      C

[777715 rows x 3 columns]


Even in this overlap, both dataframes still have multiple accounts for the same person. To understand what is going on, we should examine the credit dataframe to see if the credit history for duplicate accounts is the same or different. We will do this only with a few examples of duplicated data, as the dataset is fairly large. First, let us count the number of unique ids in both lists. 

In [181]:
print(len(set(app_df_only_including_overlap["ID"])))
print(len(set(credit_df_only_including_overlap["ID"])))

36457
36457


This is good. It tells us that each row in the condensed application data table has a unique account id at least, even if multiple accounts correspond to the same person. The condensed credit data also has these same accounts, as expected, although rows in this data are identified by both the account id and the month the account started being recorded. We next want to identify all accounts that correspond to the same person as a given account in application data. 

In [245]:
def DuplicateIDs(df, ID):
    '''Parameters are a dataframe with an ID column and a specified ID number. 
       Returns a list of IDs that have the same data as the given ID across other columns'''
    df_ID = df.loc[df["ID"]==ID]
    index = df_ID.index.values.tolist()[0]
    columns = df.columns.values.tolist()
    columns_remaining = [x for x in columns if x!='ID']
    dupe_indices = DuplicateList(df, index, columns_remaining)
    return df.loc[dupe_indices, "ID"]

Let's now write a function that will take our credit dataframe and extract the entries whose id numbers correspond to duplicates of a given id.

In [259]:
def GetDuplicateData(ID, output_df=credit_df_only_including_overlap, identifier_df=app_df_only_including_overlap):
    '''Parameters: ID is an id number. output_df and identifier_df are dataframes that have an ID column 
        with all ID values shared in common in both dataframes.
       Returns: Entries in output_df that correspond to the duplicate id_values, with duplicates identified 
        via identifier_df'''
    id_series=DuplicateIDs(identifier_df, ID)
    
    return output_df.loc[output_df["ID"].isin(id_series)]
    

Here are some examples. 

In [262]:
print(GetDuplicateData(5008804))

            ID  MONTHS_BALANCE STATUS
92938  5008804               0      C
92939  5008804              -1      C
92940  5008804              -2      C
92941  5008804              -3      C
92942  5008804              -4      C
92943  5008804              -5      C
92944  5008804              -6      C
92945  5008804              -7      C
92946  5008804              -8      C
92947  5008804              -9      C
92948  5008804             -10      C
92949  5008804             -11      C
92950  5008804             -12      C
92951  5008804             -13      1
92952  5008804             -14      0
92953  5008804             -15      X
92954  5008805               0      C
92955  5008805              -1      C
92956  5008805              -2      C
92957  5008805              -3      C
92958  5008805              -4      C
92959  5008805              -5      C
92960  5008805              -6      C
92961  5008805              -7      C
92962  5008805              -8      C
92963  50088

In [284]:
print(GetDuplicateData(app_df_only_including_overlap.loc[3, "ID"]).iloc[0:40])

            ID  MONTHS_BALANCE STATUS
92999  5008808               0      0
93000  5008808              -1      X
93001  5008808              -2      0
93002  5008808              -3      X
93003  5008808              -4      X
93004  5008809             -22      X
93005  5008809             -23      X
93006  5008809             -24      X
93007  5008809             -25      X
93008  5008809             -26      X
93009  5008810               0      C
93010  5008810              -1      C
93011  5008810              -2      C
93012  5008810              -3      C
93013  5008810              -4      C
93014  5008810              -5      C
93015  5008810              -6      C
93016  5008810              -7      C
93017  5008810              -8      C
93018  5008810              -9      C
93019  5008810             -10      C
93020  5008810             -11      C
93021  5008810             -12      C
93022  5008810             -13      C
93023  5008810             -14      C
93024  50088

In [285]:
print(GetDuplicateData(app_df_only_including_overlap.loc[3, "ID"]).iloc[40:])

            ID  MONTHS_BALANCE STATUS
93039  5008811              -3      C
93040  5008811              -4      C
93041  5008811              -5      C
93042  5008811              -6      C
93043  5008811              -7      C
93044  5008811              -8      C
93045  5008811              -9      C
93046  5008811             -10      C
93047  5008811             -11      C
93048  5008811             -12      C
93049  5008811             -13      C
93050  5008811             -14      C
93051  5008811             -15      C
93052  5008811             -16      C
93053  5008811             -17      C
93054  5008811             -18      C
93055  5008811             -19      C
93056  5008811             -20      C
93057  5008811             -21      C
93058  5008811             -22      C
93059  5008811             -23      C
93060  5008811             -24      C
93061  5008811             -25      C
93062  5008811             -26      C
93063  5008811             -27      X
93064  50088

In [290]:
print(NextNonDuplicate(app_df, 3, columns_remaining))
print(DuplicateList(app_df, 7, columns_remaining))

[7, 5008812]
[7, 8, 9]


In [291]:
print(GetDuplicateData(app_df_only_including_overlap.loc[7, "ID"]))

            ID  MONTHS_BALANCE STATUS
93075  5008812              -4      0
93076  5008812              -5      X
93077  5008812              -6      X
93078  5008812              -7      0
93079  5008812              -8      0
93080  5008812              -9      0
93081  5008812             -10      0
93082  5008812             -11      0
93083  5008812             -12      0
93084  5008812             -13      0
93085  5008812             -14      0
93086  5008812             -15      0
93087  5008812             -16      0
93088  5008812             -17      0
93089  5008812             -18      0
93090  5008812             -19      0
93091  5008812             -20      X
93092  5008813               0      0
93093  5008813              -1      X
93094  5008813              -2      X
93095  5008813              -3      0
93096  5008813              -4      0
93097  5008813              -5      0
93098  5008813              -6      0
93099  5008813              -7      0
93100  50088

These tables are suggesgtive. The different account ids in application data are linked to the same person, but correspond to different credit accounts with the bank the data was collected from. This suggests an approach to deal with the duplication:
1. Use drop_duplicate to only keep the top entry in each block corresponding to a single person.
2. Record the other account numbers of the person as a single new column in the application dataframe.
3. Use the credit data for all accounts of this person to create a new label column in the application dataframe. 